<a href="https://colab.research.google.com/github/mandananaderi/MLP_Projects_2025/blob/main/RBF_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [20]:
import keras
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from keras.layers import Layer
from tensorflow.keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

# Breast Cancer Data Loading

In [22]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
label = np.zeros((len(y), 2))
label[np.where(y == 1), 1] = 1  # Wherever y is equal to 1, put the number 1 in column 1
label[np.where(y == 0), 0] = 1  # Wherever y is equal to 0, put the number 1 in column 0

Xtrain, Xtest, y_train, y_test = train_test_split(X, label, test_size = 0.3)
scaler_X = MinMaxScaler().fit(Xtrain)
X_train, X_test = scaler_X.transform(Xtrain), scaler_X.transform(Xtest)


# Define RBF Layer

In [21]:
class RBFLayer(Layer):

  def __init__(self, units, gamma, **kwargs):
    super(RBFLayer, self).__init__(**kwargs)
    self.units = units
    self.gamma = K.cast_to_floatx(gamma)


  def build(self, input_shape):
    self.mu = self.add_weight(name='mu',
                              shape= (int(input_shape[1]), self.units),
                              initializer='uniform',
                              trainable= True)
    super(RBFLayer, self).build(input_shape)


  def call(self, inputs):
    diff = K.expand_dims(inputs)- self.mu
    l2 = K.sum(K.pow(diff, 2), axis=1)
    res = K.exp(-1 * self.gamma * l2)
    return res


  def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

# Simple MLP Training


In [12]:
model = Sequential()
model.add(Dense(50, activation = 'sigmoid'))
model.add(Dense(2, activation = 'sigmoid'))
opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer = opt, loss=loss, metrics =['binary_crossentropy'])
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 5)

Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - binary_crossentropy: 0.7685 - loss: 0.7685 - val_binary_crossentropy: 0.6664 - val_loss: 0.6664
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_crossentropy: 0.6593 - loss: 0.6593 - val_binary_crossentropy: 0.6535 - val_loss: 0.6535
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_crossentropy: 0.6657 - loss: 0.6657 - val_binary_crossentropy: 0.6486 - val_loss: 0.6486
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_crossentropy: 0.6363 - loss: 0.6363 - val_binary_crossentropy: 0.6446 - val_loss: 0.6446
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_crossentropy: 0.6474 - loss: 0.6474 - val_binary_crossentropy: 0.6404 - val_loss: 0.6404
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_crossentropy: 0.6440 - loss: 0.6440 - val_binary_crossentropy: 0.6365 - val_loss: 0.6365
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_crossentropy: 0.6162 - loss: 0.6162 - val_bina


# Evaluation

In [13]:
y_pre = model.predict(X_test)
confusion_matrix(np.argmax(y_pre, axis=1), np.argmax(y_test, axis = 1))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


array([[ 57,   3],
       [  7, 104]])

# RBF Neural Netrowk Training


In [23]:
model = Sequential()
model.add(RBFLayer(50, 0.15, name = 'RBF', trainable = True))
model.add(keras.layers.Dense(2, activation = 'sigmoid'))
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer = opt, loss=loss, metrics = ['binary_crossentropy'])
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 200, batch_size = 5)

Epoch 1/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - binary_crossentropy: 0.6166 - loss: 0.6166 - val_binary_crossentropy: 0.5691 - val_loss: 0.5691
Epoch 2/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_crossentropy: 0.5980 - loss: 0.5980 - val_binary_crossentropy: 0.5275 - val_loss: 0.5275
Epoch 3/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - binary_crossentropy: 0.5719 - loss: 0.5719 - val_binary_crossentropy: 0.4988 - val_loss: 0.4988
Epoch 4/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_crossentropy: 0.5430 - loss: 0.5430 - val_binary_crossentropy: 0.4720 - val_loss: 0.4720
Epoch 5/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_crossentropy: 0.4750 - loss: 0.4750 - val_binary_crossentropy: 0.4496 - val_loss: 0.4496
Epoch 6/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_crossentropy: 0.4316 - loss: 0.4316 - val_binary_crossentropy: 0.4470 - val_loss: 0.4470
Epoch 7/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - binary_crossentropy: 0.4535 - loss: 0.4535 - val_bina

# Evaluation

In [24]:
y_pre = model.predict(X_test)
confusion_matrix(np.argmax(y_pre, axis=1), np.argmax(y_test, axis = 1))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


array([[ 57,   3],
       [  3, 108]])